In [23]:
import illustris_python.groupcat as gc
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import numpy as np
import abacus_cosmos.Halos as ach
from astropy.table import Table

%matplotlib inline

In [55]:
basePath = "/Users/forero/github/abacus/data/AbacusCosmos_720box_04_FoF_halos_z0.100/"
halo_data = ach.read_halos_FoF(basePath)
halo_table = Table(halo_data)

In [56]:
print(halo_table['pos'].min())

-359.99994


In [57]:
hubble = 0.67
BoxSize = 720.0

In [58]:
halo_table['pos'] = halo_table['pos']+BoxSize/2.0
print(halo_table['pos'].min())

6.1035156e-05


In [59]:
halo_table[:10]

id,subsamp_start,subsamp_len,N,subhalo_N [4],pos [3],vel [3],sigma_v [3],r25,r50,r75,r90,vcirc_max,rvcirc_max,subhalo_pos [3],subhalo_vel [3],subhalo_sigma_v [3],subhalo_r25,subhalo_r50,subhalo_r75,subhalo_r90,subhalo_vcirc_max,subhalo_rvcirc_max
int64,uint64,uint32,uint32,uint32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
33242377525485,0,6514,65785,43076 .. 272,68.90936 .. 254.36768,527.46204 .. -306.52667,913.1875 .. 641.1195,0.47896388,0.95461166,1.4652847,1.9062219,1232.1814,1.133729,-291.06955 .. -105.6578,531.79333 .. -320.0819,913.2037 .. 641.2682,0.48745063,0.95618564,1.4646742,1.9327941,1229.9573,0.79291576
33242377525486,6514,5462,54295,39443 .. 136,152.90689 .. 353.71448,-203.81299 .. 237.39218,947.0979 .. 719.2114,0.36945322,0.6556443,1.0200204,1.4969106,1347.0299,0.66329706,-207.13239 .. -6.319993,-198.0364 .. 264.49124,947.3582 .. 719.2907,0.3634097,0.65406406,1.0204002,1.4948139,1349.2322,0.62759477
33242377525487,11976,4934,48930,33131 .. 310,25.339355 .. 71.402954,-37.641155 .. 217.23412,896.5138 .. 647.6147,0.48626307,0.74851435,1.1311457,1.5374379,1215.3206,0.9066862,-334.61246 .. -288.65997,-97.796524 .. 206.15282,899.4202 .. 648.18024,0.4744195,0.7499747,1.1342478,1.5676634,1215.1339,0.89925796
33242377525488,16910,3953,39453,24467 .. 241,117.45816 .. 263.1902,-80.68883 .. -354.3293,727.8095 .. 500.5693,0.61909246,0.92649895,1.3504988,1.7689579,987.9422,1.1386218,-242.57126 .. -96.76999,-114.792206 .. -306.4037,731.40515 .. 501.61682,0.53929186,0.92297125,1.3495759,1.8150662,989.54236,1.211195
33242377525489,20863,3660,36239,24363 .. 117,0.17297363 .. 80.42062,286.564 .. -178.37665,757.8137 .. 581.61646,0.3430936,0.6619118,1.0475227,1.3595113,1095.1309,0.6957455,-359.82068 .. -279.61578,296.14957 .. -192.95085,757.97723 .. 581.9728,0.33681205,0.6709805,1.0389625,1.3587822,1095.3732,0.45118648
33242377525490,24523,3171,31997,24187 .. 102,152.81091 .. 487.31912,267.2844 .. -460.34653,810.4758 .. 518.2186,0.41604078,0.78409654,1.1286488,1.4011918,965.0367,1.1532272,-207.19937 .. 127.31631,273.6985 .. -468.54953,811.1688 .. 518.3678,0.4179683,0.7863768,1.1281159,1.4001321,965.9276,1.148427
33242377525491,27694,2968,29822,22386 .. 90,27.395569 .. 165.91907,92.99687 .. -215.39044,923.4031 .. 499.8083,0.31365192,0.54878795,0.87086356,1.1481982,1092.0194,0.5896845,-332.62054 .. -194.05328,89.58194 .. -211.28677,923.4254 .. 499.88705,0.30305943,0.54301673,0.86891997,1.1624866,1098.32,0.49110937
33242377525492,30662,2873,28794,20800 .. 190,50.763397 .. 610.12134,-448.97797 .. 134.78462,790.5677 .. 558.58936,0.29329008,0.5304205,0.86879677,1.1874532,1091.7693,0.5160773,-309.25833 .. 250.07993,-462.22513 .. 143.15332,790.5681 .. 558.81,0.28870782,0.53103656,0.86709386,1.1915562,1093.1848,0.47086686
33242377525493,33535,2465,24615,16832 .. 59,154.29433 .. 377.3882,-63.092236 .. 33.133675,658.73 .. 530.7746,0.31509176,0.57393515,0.9139683,1.2502626,975.7722,0.69975984,-205.71552 .. 17.431177,-57.846317 .. 15.195254,658.7556 .. 531.0555,0.29599875,0.55468965,0.9181996,1.2903208,990.305,0.6197941


In [60]:
print(np.min(halo_table['vcirc_max']))

48.942024


In [61]:
ii = halo_table['vcirc_max']>200 # in units of km/s
print(np.count_nonzero(ii))

682062


In [62]:
S_pos = halo_table['pos'][ii]
S_vel = halo_table['vel'][ii]
S_vmax = halo_table['vcirc_max'][ii]
S_stellar_mass = np.zeros(np.count_nonzero(ii))
S_parent_fof = np.zeros(np.count_nonzero(ii))
n_S = len(S_pos)
print(n_S)

682062


In [63]:
#pad boxes around the S3 positions to mimic periodic boundary conditions
S_pad_pos = S_pos.copy()
S_pad_vel = S_vel.copy()
S_pad_vmax = S_vmax.copy()
S_pad_stellar_mass = S_stellar_mass.copy()
S_pad_fof = S_parent_fof.copy()
S_pad_id = np.arange(n_S)
for i in [0]:
    for j in [0]:
        for k in [0]:
            new_pos = S_pos.copy()
            if(i):
                new_pos[:,0] = new_pos[:,0] + i*BoxSize
            if(j):
                new_pos[:,1] = new_pos[:,1] + j*BoxSize
            if(k):
                new_pos[:,2] = new_pos[:,2] + k*BoxSize
                
            if((i!=0) | (j!=0) | (k!=0)):
                S_pad_pos = np.append(S_pad_pos, new_pos, axis=0)
                S_pad_vel = np.append(S_pad_vel, S_vel, axis=0)
                S_pad_vmax = np.append(S_pad_vmax, S_vmax)
                S_pad_stellar_mass = np.append(S_pad_stellar_mass, S_stellar_mass)
                S_pad_id = np.append(S_pad_id, np.arange(n_S))
                S_pad_fof = np.append(S_pad_fof, S_parent_fof)

In [64]:
nbrs_S = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(S_pad_pos)
dist_S, ind_S = nbrs_S.kneighbors(S_pad_pos)
print(S_pad_pos.shape)
print(dist_S.shape)

(682062, 3)
(682062, 20)


In [65]:
neighbor_index = ind_S[:,1]
neighbor_list = ind_S[:,2:]
print(np.shape(neighbor_list))

n_pairs = 0

halo_A_id = np.empty((0), dtype=int)
halo_B_id = np.empty((0), dtype=int)

for i in range(n_S):
#for i in range(10):
    l = neighbor_index[neighbor_index[i]]% n_S
    j = neighbor_index[i] % n_S
    
    other_j = neighbor_list[i,:] % n_S
    other_l = neighbor_list[neighbor_index[i],:] % n_S
    
    if((i==l) & (not (j in halo_A_id)) & (not (i in halo_B_id))): # first check to find mutual neighbors
        if((dist_S[i,1] > 0.7)): #check on the distance between the two galaxies
            #print('distancia')
            vmax_i = S_pad_vmax[i]
            vmax_j = S_pad_vmax[j]
            vmax_limit = min([vmax_i, vmax_j])
                
            pair_d = dist_S[i,1] # This is the current pair distance
            dist_limit = pair_d * 3.0 # exclusion radius for massive structures
            
            massive_close_to_i = any((dist_S[i,2:]<dist_limit) & (S_pad_vmax[other_j] >= vmax_limit))
            massive_close_to_j = any((dist_S[j,2:]<dist_limit) & (S_pad_vmax[other_l] >= vmax_limit))
            if((not massive_close_to_i) & (not massive_close_to_j)): # check on massive structures inside exclusion radius
                n_pairs = n_pairs+ 1
                halo_A_id = np.append(halo_A_id, int(i))
                halo_B_id = np.append(halo_B_id, int(j))
                    #print(pair_d)
print(n_pairs)

(682062, 18)
29161


In [66]:
halo_A = np.array([S_pad_fof[halo_A_id], np.zeros(len(S_pad_fof[halo_A_id])), 
                   S_pad_stellar_mass[halo_A_id], S_vmax[halo_A_id],
                   S_pos[halo_A_id,0], S_pos[halo_A_id,1], S_pos[halo_A_id,2],
                   S_vel[halo_A_id,0], S_vel[halo_A_id,1], S_vel[halo_A_id,2]], )
np.savetxt('../data/vmax_selected/summary_halo_A_box_04.txt', halo_A.T, 
        fmt="%d %f %f %f %f %f %f %f %f %f", 
           header = "FOF_ID M_FOF[10^10 Msun/h] Mstar[10^10 Msun/h] Vmax [km/s] Pos_xyz [kpc/h] Vel_xyz [km/s]")

halo_B = np.array([S_pad_fof[halo_B_id], np.zeros(len(S_pad_fof[halo_B_id])), 
                   S_pad_stellar_mass[halo_B_id], S_vmax[halo_B_id],
                   S_pos[halo_B_id,0], S_pos[halo_B_id,1], S_pos[halo_B_id,2],
                   S_vel[halo_B_id,0], S_vel[halo_B_id,1], S_vel[halo_B_id,2]])
np.savetxt('../data/vmax_selected/summary_halo_B_box_04.txt', halo_B.T, 
        fmt="%d %f %f %f %f %f %f %f %f %f", 
           header = "FOF_ID M_FOF[10^10 Msun/h] Mstar[10^10 Msun/h] Vmax [km/s] Pos_xyz [kpc/h] Vel_xyz [km/s]")

halo_C = np.array([S_pad_fof, np.zeros(len(S_pad_fof)), 
                   S_pad_stellar_mass, S_vmax,
                   S_pos[:,0], S_pos[:,1], S_pos[:,2],
                   S_vel[:,0], S_vel[:,1], S_vel[:,2]])
np.savetxt('../data/vmax_selected/summary_halo_general_box_04.txt', halo_C.T, 
        fmt="%d %f %f %f %f %f %f %f %f %f", 
           header = "FOF_ID M_FOF[10^10 Msun/h] Mstar[10^10 Msun/h] Vmax [km/s] Pos_xyz [kpc/h] Vel_xyz [km/s]")